In [1]:
import numpy as np
import pandas as pd
import random

from pgmpy.models import BayesianModel
from pgmpy.inference import VariableElimination

from memo import memo

In [2]:
label = {'interest': ["sports", "politics"],
         'connect_type': ["acquaintance", "close friend", "family"],
    }

In [3]:
class Person(object):
    def __init__(self):
        self.interest = self.getInterest()
    
    def getInterest(self):
        self.interest = random.randint(0, 1)
        return self.interest

In [4]:
person1 = Person()
print(person1.interest)

1


In [5]:
class Post(object):
    def __init__(self, poster):
        self.poster = poster
        self.topic = self.getTopic()
        
    def getTopic(self):
        isPoster = random.choices([0,1], weights = [1,9], k=1)[0]
        if isPoster:
            self.topic = self.poster.interest
        else:
            self.topic = random.randint(0, 1)
        return self.topic

In [6]:
post1 = Post(person1)
print(post1.topic)

1


In [7]:
class Connection(object):
    def __init__(self, person1, person2):
        self.person1 = person1
        self.person2 = person2
        self.type = self.getType()[0]
        
    def getType(self):
        self.type = random.choices([0,1,2], k=1)
        return self.type

@memo
def generateConnection(person1, person2):
    return Connection(person1, person2)

In [8]:
person1 = Person()
person2 = Person()

In [9]:
generateConnection(person1, person2).type

2

In [10]:
class Comment(object):
    def __init__(self, post, commenter):
        self.post = post
        self.commenter = commenter
        self.isTopicMatch = post.topic == commenter.interest
        self.connectionType = generateConnection(post.poster, commenter).type
        self.constraint = self.getConstraint()
        
    def getConstraint(self):
        if self.isTopicMatch: 
            self.constraint = 1
        elif self.connectionType == 2:
            self.constraint = 0.8
        elif self.connectionType == 1:
            self.constraint = 0.5
        else:
            self.constraint = 0.1
        return self.constraint

In [11]:
amy = Person()
brian = Person()
cheryl = Person()

post1 = Post(amy)
post2 = Post(brian)
post3 = Post(amy)

comment1 = Comment(post1, brian)
comment2 = Comment(post1, cheryl)
comment3 = Comment(post2, amy)
comment4 = Comment(post3, cheryl)

In [12]:
### Generate instance
interest = []
topic = []
c_interest = []
match = []
relationship = []
for i in range(1000):
    Amy = Person()
    Brian = Person()
    post1 = Post(Amy)
    comment1 = Comment(post1, Brian)
    rlt = generateConnection(Amy,Brian).type
    interest.append(Amy.interest)
    topic.append(post1.topic)
    match.append(comment1.constraint)
    c_interest.append(Brian.interest)
    relationship.append(rlt)

In [13]:
### fit model and using VariableElimination to calculate the posterior probability 
d = {'interest':interest, 'topic':topic, "c_interest":c_interest, 'match':match, 'relationship':relationship}
values = pd.DataFrame(data = d)
model = BayesianModel([('interest', 'topic'), ('topic', 'match'), ('c_interest', 'match'),('relationship','interest'),('relationship','c_interest')])
model.fit(values)
inference = VariableElimination(model)

In [15]:
senario_1 = inference.query(['interest'],evidence={'relationship':2,'topic':0},show_progress=False)
senario_1_c = inference.query(['c_interest','match'],evidence={'relationship':2,'topic':0},show_progress=False)

In [16]:
print("Amy(Poster)'s interest given post topic = 'sports', relationtype = 'family'.\n",senario_1)

Amy(Poster)'s interest given post topic = 'sports', relationtype = 'family'.
 +-------------+-----------------+
| interest    |   phi(interest) |
+=============+=================+
| interest(0) |          0.9633 |
+-------------+-----------------+
| interest(1) |          0.0367 |
+-------------+-----------------+


In [17]:
print("Brian(Commenter)'s interest given post topic = 'sports', relationtype = 'family'.\n",senario_1_c)

Brian(Commenter)'s interest given post topic = 'sports', relationtype = 'family'.
 +------------+---------------+-------------------------+
| match      | c_interest    |   phi(match,c_interest) |
+============+===============+=========================+
| match(0.1) | c_interest(0) |                  0.0000 |
+------------+---------------+-------------------------+
| match(0.1) | c_interest(1) |                  0.1768 |
+------------+---------------+-------------------------+
| match(0.5) | c_interest(0) |                  0.0000 |
+------------+---------------+-------------------------+
| match(0.5) | c_interest(1) |                  0.1619 |
+------------+---------------+-------------------------+
| match(0.8) | c_interest(0) |                  0.0000 |
+------------+---------------+-------------------------+
| match(0.8) | c_interest(1) |                  0.1597 |
+------------+---------------+-------------------------+
| match(1.0) | c_interest(0) |                  0.5016 |
+----

In [18]:
senario_2 = inference.query(['interest'],evidence={'topic':1,'match':1},show_progress=False)
senario_2_c = inference.query(['c_interest'],evidence={'topic':1,'match':1},show_progress=False)

In [19]:
print("Amy(Poster)'s interest given post topic = 'politics', topic_match = 1.\n",senario_2)

Amy(Poster)'s interest given post topic = 'politics', topic_match = 1.
 +-------------+-----------------+
| interest    |   phi(interest) |
+=============+=================+
| interest(0) |          0.0493 |
+-------------+-----------------+
| interest(1) |          0.9507 |
+-------------+-----------------+


In [20]:
print("Brian(Poster)'s interest given post topic = 'politics', topic_match = 1.\n",senario_2_c)

Brian(Poster)'s interest given post topic = 'politics', topic_match = 1.
 +---------------+-------------------+
| c_interest    |   phi(c_interest) |
+===============+===================+
| c_interest(0) |            0.0000 |
+---------------+-------------------+
| c_interest(1) |            1.0000 |
+---------------+-------------------+
